# Keras tutorial - Распознавание эмоций на изображениях лиц

В рамках данной лабораторной работы:
1. Изучите Keras, высокоуровневый API для нейронных сетей (programming framework), написанный на Python и способный работать поверх нескольких низкоуровневых фреймворков, включая TensorFlow и CNTK. 
2. Построите алгоритм глубокого обучения.

#### Зачем использовать Keras? 

* Keras был разработан инженрами и специалистами в области глубокого обучения для быстрого построения моделей
* TensorFlow является фреймворком высокого уровня, Keras является более высокоуровневым фреймворком и предоставляет дополнительные абстракции. 
* Способность перейти от идеи к результату с наименьшей возможной задержкой является ключом к поиску хороших моделей. 
* Однако Keras является более ограничительным, чем фреймворки более низкого уровня, поэтому есть некоторые очень сложные модели, которые вы все равно будете реализовывать в TensorFlow , а не в Keras. 
* Тем не менее, Keras будет отлично работать для многих распространенных моделей.

`Данный материал опирается и использует материалы курса Deep Learning от организации deeplearning.ai`
 
 Ссылка на основной курс (для желающих получить сертификаты): https://www.coursera.org/specializations/deep-learning

## 1 - Пакеты/Библиотеки

Первоначально необходимо запустить ячейку ниже, чтобы импортировать все пакеты, которые вам понадобятся во время лабораторной работы.

In [ ]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

**Заметка**: Мы ранее импортировали множество функций из Keras, которые можно использовать в своём коде. 
Например `X = Input(... )` или `X = Zeropadding2n(... )`. 

Другими словами, в отличие от TensorFlow, нет необходимости создавать граф, а затем делать отдельный `sess.run()` для вычисления этих переменных.

## 1 - Отслеживание эмоций

* Необходимо создать приложение, которое классифицирует эмоции человека на "happy" и "not happy".

Пример датасета на изображениие:
<img src="images/face_images.png" style="width:550px;height:250px;">

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Нормализация
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

## 2 - Построение модели в Keras

В Keras можно быстро делать прототип моделей, вот пример одной из них:
```python
def model(input_shape):
    """
    input_shape:  (height, width, channels) - tuple.  
    """
    
    # Определение входной tensor с размером input_shape (размер изображения).
    X_input = Input(input_shape)

    # Zero-Padding: заполнение X_input нулями по краям изображения
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    # FLATTEN X (конвертирование в вектор) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Создание экземпляра модели Keras, вы будете использовать этот экземпляр для обучения тестирования модели.
    model = Model(inputs = X_input, outputs = X, name='HappyModel')
    
    return model
```

#### Наименование переменных

* Note that Keras uses a different convention with variable names than we've previously used with numpy and TensorFlow. 
* Instead of creating unique variable names for each step and each layer, such as 
* Обратите внимание, что Keras использует другое соглашение с именами переменных, чем ранее использовался с numpy и TensorFlow. 
* Вместо создания уникальных имен переменных для каждого шага и каждого слоя, таких как
```
X = ...
Z1 = ...
A1 = ...
```
* Keras перезаписывает переменную на каждом шаге:
```
X = ...
X = ...
X = ...
```
* Исключение составляет `X_input`, который необходимо держать отдельно, так как он используется позже.

#### Объекты, как функции
* Обратите внимание, что в каждом операторе есть две пары скобок. Например:
```
X = ZeroPadding2D((3, 3))(X_input)
```
* Первая - это вызов конструктора, который создает объект (ZeroPadding2D).
* В Python объекты могут быть вызваны как функции. Более подробно об этом можно прочитать в блоге:[Python Pandemonium](https://medium.com/python-pandemonium/function-as-objects-in-python-d5215e6d1b0d).
```
ZP = ZeroPadding2D((3, 3)) # ZP, который вызывается как функция
X = ZP(X_input) 
```

**Упражнение**: Реализовать функцию `HappyModel()`.  
* Начните с реализации модели, используя предложенную архитектуру, и выполните оставшуюся часть этого задания, используя ее в качестве исходной модели. 
* Позже вернитесь и попробуйте другие архитектуры. 
* Например, можете черпать вдохновение из приведенной выше модели, но затем изменять сетевую архитектуру и гиперпараметры по своему усмотрению.
* Можно использовать также другие функции активации `AveragePooling2D()`, `GlobalMaxPooling2D()`, `Dropout()`. 

In [ ]:
# GRADED FUNCTION: HappyModel

def HappyModel(input_shape):
    """
    Реализация HappyModel.
    
    Arguments:
    input_shape -- размер изображений и входного датасета
        (height, width, channels) - tuple.  
    Returns:
        model -- Model() объект Keras


    """
    
    ### НАЧАЛО ВАШЕГО КОДА ###
    
    
    ### КОНЕЦ ВАШЕГО КОДА ###
    
    return model

You have now built a function to describe your model. To train and test this model, there are four steps in Keras:
После построения самой модели необходимо её обучить и протестировать. Необходимо сделать четыре шага:
1. Создать модель, вызвав функцию выше  

2. Скомпилировать модель, вызвав - `model.compile(optimizer = "...", loss = "...", metrics = ["accuracy"])`  

3. Обучить модель,вызвав - `model.fit(x = ..., y = ..., epochs = ..., batch_size = ...)`  

4. Протестировать модель - `model.evaluate(x = ..., y = ...)`  

Более подробно про `model.compile()`, `model.fit()`, `model.evaluate()` в официальной документации [Keras documentation](https://keras.io/models/model/).

#### Step 1: Создание модели
**Подсказка**:  
`X_train.shape[1:]` как `input_shape`.

In [ ]:
### НАЧАЛО ВАШЕГО КОДА ### (1 line)
happyModel = None
### КОНЕЦ ВАШЕГО КОДА ###

#### Step 2: Компиляция модели

**Подсказка**:  
Оптимизаторы - [optimizers](https://keras.io/optimizers/)  
В лабораторной работе поставлена задача бинарной классификации.  Можно использовать функцию потерь `'binary_cross_entropy'` - [losses](https://keras.io/losses/)

In [ ]:
### НАЧАЛО ВАШЕГО КОДА ### (1 строка кода)
None
### КОНЕЦ ВАШЕГО КОДА ###

#### Step 3: Обучение модели

**Подсказка**:  
Используйте `'X_train'`, `'Y_train'` переменные.

**Заметка**: Если вы снова запустите `fit()`, то `model` будет продолжать обучаться с уже изученными параметрами вместо их повторной инициализации.

In [ ]:
### НАЧАЛО ВАШЕГО КОДА ### (1 строка кода)
None
### КОНЕЦ ВАШЕГО КОДА ###

#### Step 4: Оценка модели
**Подсказка**:  
Используйте `'X_test'` и `'Y_test'` переменные.

In [ ]:
### НАЧАЛО ВАШЕГО КОДА ### (1 строка кода)
preds = None
### КОНЕЦ ВАШЕГО КОДА ###
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

#### Ожидаемое качество
Если функция `happyModel()` работает корректно, ее точность должна быть лучше, чем случайное угадывание (точность выше 50%).

Чтобы дать вам точку сравнения, наша модель получает около **95% точности на тесте в 40 эпохах** (и 99% точности на обучении) с mini-batch размером 16 и оптимизатором "adam".

#### Tips for improving your model

If you have not yet achieved a very good accuracy (>= 80%), here are some things tips:

- Use blocks of CONV->BATCHNORM->RELU such as:
```python
X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv0')(X)
X = BatchNormalization(axis = 3, name = 'bn0')(X)
X = Activation('relu')(X)
```
until your height and width dimensions are quite low and your number of channels quite large (≈32 for example).  
You can then flatten the volume and use a fully-connected layer.
- Use MAXPOOL after such blocks.  It will help you lower the dimension in height and width.
- Change your optimizer. We find 'adam' works well. 
- If you get memory issues, lower your batch_size (e.g. 12 )
- Run more epochs until you see the train accuracy no longer improves. 

**Note**: If you perform hyperparameter tuning on your model, the test set actually becomes a dev set, and your model might end up overfitting to the test (dev) set. Normally, you'll want separate dev and test sets.  The dev set is used for parameter tuning, and the test set is used once to estimate the model's performance in production.

#### Советы по улучшению вашей модели

Если вы еще не достигли очень хорошей точности (>=80%), вот некоторые советы:

- Используйте блоки CONV->BATCHNORM->RELU, такие как:
```python
X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv0')(X)
X = BatchNormalization(axis = 3, name = 'bn0')(X)
X = Activation('relu')(X)
```
до тех пор, пока ваши размеры по высоте и ширине не станут совсем низкими, а количество каналов - большим (например, ≈32).Затем вы можете выровнять объем и использовать fully-connected layer.
- Используйте MAXPOOL после таких блоков. Это поможет вам уменьшить размер по высоте и ширине.
- Измените свой оптимизатор.
- Если у вас возникли проблемы с памятью, уменьшите размер пакета (например, 12)
- Запускайте больше эпох, пока не увидите, что точность на обучении больше не улучшается.

## 3 - Выводы

## Ключевые моменты, которые нужно запомнить
- Keras - это инструмент, который рекомендуется для быстрого прототипирования. Это позволяет быстро опробовать различные архитектуры моделей.
- Помните четыре шага в Keras: 
1. Создавать 
2. Компилировать 
3. Fit/Train 
4. Evaluate/Test

## 4 - Тестирование на своих изображениях

In [ ]:
### НАЧАЛО ВАШЕГО КОДА ###
img_path = 'images/my_image.jpg'
### КОНЕЦ ВАШЕГО КОДА ###
img = image.load_img(img_path, target_size=(64, 64))
imshow(img)

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print(happyModel.predict(x))

## 5 - Другие полезные функции

Две другие основные функции Keras, которые вы найдете полезными:: 
- `model.summary()`: выводит сведения о ваших слоях в таблицу с размерами входов/выходов
- `plot_model()`: строит ваш график в красивом макете. Вы даже можете сохранить его как ".png " с помощью SVG().

In [ ]:
happyModel.summary()

In [ ]:
plot_model(happyModel, to_file='HappyModel.png')
SVG(model_to_dot(happyModel).create(prog='dot', format='svg'))